### Работа с пропущенными значениями (NaN)

Иногда при работе с данными можно столкнуться с такой проблемой: в данных есть пропущенные значения, но удалять из таблицы строки с пропущенными значениями не хочется, поскольку в этих строках есть также заполненные ячейки с ценной информацией. 

Особенно это актуально при работе с большими таблицами, где очень много показателей, но при этом значительная доля этих показателей специфическая, например, рассчитывается не для каждого года и страны, что может привести к ситуации, когда в каждой строке таблицы есть хотя бы одна ячейка с пропущенным значением. Очевидно, что в таком случае при обычном удалении строк с пропущенными значениями мы останемся с пустой таблицей! Поэтому часто в таких ситуациях пропущенные значения не удаляют, а заполняют.

Если данные не содержат нулевых значений, то пропущенные значения можно заменить нулями. Для этого достаточно применить к датафрейму pandas метод `.fillna()`:

In [3]:
import pandas as pd
import numpy as np

# создадим маленький датафрейм
ages = pd.DataFrame({'age':[24, 25, np.nan, 29], 
                   'income':[20000, np.nan, 26000, 30000],
                   'children':[2, 1, 3, np.nan]})
ages

,age,income,children
0,24.0,20000.0,2.0
1,25.0,NaN,1.0
2,NaN,26000.0,3.0
3,29.0,30000.0,NaN


In [4]:
# заполним всё нулями
ages.fillna(0)

,age,income,children
0,24.0,20000.0,2.0
1,25.0,0.0,1.0
2,0.0,26000.0,3.0
3,29.0,30000.0,0.0


При работе с опросами часто пропущенные значения заменяют специальными кодами. Часто эти коды сильно отличаются от «обычных» значений. Например, ответы на вопрос о некотором утверждении имеют метки:

*1 – абсолютно не согласен с утверждением, 2 – не согласен с утверждением, 3 – затрудняюсь ответить, 4 – согласен с утверждением, 5 – абсолютно согласен с утверждением.*

Тогда в случае отсутствия ответа на вопрос можно поставить значение 98 или 99, потому что встретить такой ответ при обычном сценарии развития событий не получится, сам вопрос не позволит.

Однако иногда недостаточно заполнить пустые ячейки одним значением. Ведь когда мы заполняем их одним значением, мы все равно теряем информацию – нулевые значения или закодированные особым образом мы потом отфильтруем и не будем использовать в дальнейшем анализе. Если не хочется действовать так радикально, пропущенные значения можно заполнить средним или медианным значением по столбцу. При добавлении значения, равного среднему или медиане, распределение данных, а также среднее и медианное значения практически не меняются. Если разница между средним и медианой небольшая, распределение не является сильно скошенным вправо или влево, нет нехарактерных значений – выбросов, то нет большой разницы, каким значением – средним или медианным – заполнять. Если разница довольно большая, то лучше заполнять медианным значением, поскольку в таком случае оно более адекватно отражает середину распределения.

Для заполнения средним значением или медианой тоже подойдет метод `.fillna()`.

In [9]:
# вспомним, как считается среднее по столбцам
ages.mean()

pandas.core.series.Series

In [8]:
# подставим .mean() 
# могли бы взять .median()
ages.fillna(ages.mean())

,age,income,children
0,24.0,20000.000000,2.0
1,25.0,25333.333333,1.0
2,26.0,26000.000000,3.0
3,29.0,30000.000000,2.0


Если почитать документацию, то можно заметить, что метод `.fillna()` принимает на вход словарь, последовательность pandas *Series* или датафрейм pandas. В примере выше у нас был *Series*, но можно было бы поставить другие значения и оформить их в виде словаря:

In [10]:
# заполним столбец age средним
# столбец income – медианой
# столбец children – нулями

ages.fillna({'age': ages['age'].mean(), 
             'income': ages['income'].median(), 
             'children': 0})

,age,income,children
0,24.0,20000.0,2.0
1,25.0,26000.0,1.0
2,26.0,26000.0,3.0
3,29.0,30000.0,0.0


Если содержательно данные позволяют заполнить пропущенную ячейку значением из того же столбца ячейкой выше (например, нет значения дохода за текущий год, но его можно заполнить значением дохода за прошлый год), то можно воспользоваться методом `ffill` (от *forward fill* – заполнение вперед) и указать специальный аргумент `method`:

In [11]:
# до заполнения
ages

,age,income,children
0,24.0,20000.0,2.0
1,25.0,NaN,1.0
2,NaN,26000.0,3.0
3,29.0,30000.0,NaN


In [12]:
# вместо NaN в строке 2 стоит значение из строки 1 (age)
# вместо NaN в строке 1 стоит значение из строки 0 (income)
# вместо NaN в строке 3 стоит значение из строки 2 (children)

ages.fillna(method='ffill')

,age,income,children
0,24.0,20000.0,2.0
1,25.0,20000.0,1.0
2,25.0,26000.0,3.0
3,29.0,30000.0,3.0


Аналогичным способом можно заполнить пропущенную ячейку значением из того же столбца ячейкой ниже – метод `bfill` (от *backward fill* – заполнение назад):

In [13]:
ages.fillna(method='bfill')

,age,income,children
0,24.0,20000.0,2.0
1,25.0,26000.0,1.0
2,29.0,26000.0,3.0
3,29.0,30000.0,NaN


В примере выше в столбце *children* в последней строке остался `NaN`, так как ниже строки нет, и брать значение для заполнения неоткуда.

Больше информации можно найти в [документации](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) по методу `.fillna()`. 

Если хочется узнать про более продвинутые способы заполнения пропущенных значений (для понимания требуются знания статистики и моделирования), можно [почитать](https://www.theanalysisfactor.com/multiple-imputation-in-a-nutshell/) про множественную импутацию и ее [реализацию](https://scikit-learn.org/stable/modules/impute.html) в Python.